# 서울특별시 다산콜센터(☎120)의 주요 민원
* 서울특별시 다산콜센터(☎120)의 주요 민원(자주 묻는 질문)에 대한 답변정보
* https://opengov.seoul.go.kr/civilappeal/list

In [1]:
# 필요한 도구를 불러온다.
# 파이썬에서 사용할 수 있는 엑셀과 유사한 데이터분석 도구
# 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러옴
# request로 가져온 웹사이트의 html 태그를 찾기위해 사용
# 간격을 두고 가져오기 위해 사용

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

In [71]:
# 120 다산 콜센터의 첫 페이지를 먼저 불러와 크롤링할 내용을 봅니다.
base_url = "https://opengov.seoul.go.kr/civilappeal/list?items_per_page=50&page=1"
print(base_url)

https://opengov.seoul.go.kr/civilappeal/list?items_per_page=50&page=1


In [72]:
# pd.read_html 을 통해 해당 URL의 table 정보를 읽어옵니다.
# table
table = pd.read_html(base_url, encoding='utf-8')
print(type(table),len(table))
table[0][:4]

<class 'list'> 1


,번호,제목,생산일,조회수
0,2470,다자녀가정 실내 바닥매트 지원,2021-08-17,155
1,2469,[서울산업진흥원] 서울메이드란?,2021-06-29,675
2,2468,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,712
3,2467,마포 뇌병변장애인 비전센터,2021-03-12,840


## 상세정보를 위한 링크정보 수집
* get : 필요한 데이터를 Query String 에 담아 전송
* post : 전송할 데이터를 HTTP 메시지의 Body의 Form Data에 담아 전송

* get 과 post 여부는 브라우저의 네트워크 탭의 Headers > Request Method 를 통해 확인

In [73]:
# 웹페이지의 결과를 받아옵니다.
# response

header = {'user_agent':'Mozilla/5.0'}
r = requests.get(base_url, headers=header)
r

<Response [200]>

In [74]:
# status_code를 통한 응답코드 확인 200 == OK
r.status_code

200

## BeautifulSoup 을 통해 html 페이지를 읽기 쉽게 만들기

In [75]:
# html 태그를 파싱해 올 수 있도록 합니다.
# html
html = bs(r.text,'lxml')
html

<!-- THEME DEBUG --><!-- THEME HOOK: 'html' --><!-- FILE NAME SUGGESTIONS:
   * html--node--2672712.html.twig
   * html--node--%.html.twig
   * html--node.html.twig
   x html.html.twig
--><!-- BEGIN OUTPUT from 'themes/web/templates/layout/html.html.twig' --><!DOCTYPE html>
<html lang="ko">
<head>
<!-- <meta charset="UTF-8"> -->
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>목록 &gt; 120주요질문 &gt; 시민소통 &gt; 정보소통광장</title>
<meta content="목록" name="description"/>
<meta content="website" property="og:type"/>
<meta content="목록 &gt; 120주요질문 &gt; 시민소통 &gt; 정보소통광장" property="og:title"/>
<meta content="목록" property="og:description"/>
<meta content="://opengov.seoul.go.kr/opengov.jpg" property="og:image"/>
<meta content="://opengov.seoul.go.kr/civilappeal/list" property="og:url"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="목록 &gt; 120주요질문 &gt; 시민소통 &gt; 정보소통광장" name="twitter:title"

## 상세 정보 수집을 위한 링크 정보 찾기

In [76]:
# BeautifulSoup 의 select 기능을 통한 링크 태그 찾기
# a 태그 안의 상세 페이지 접근을 위한 링크 번호 수집
# a_list
# a_link_no
# content > div > div.view-content > div > table > tbody > tr:nth-child(1) > td.data-title.aLeft > a
#content > div > div.view-content > div > table > tbody > tr:nth-child(1) > td.data-title.aLeft > a
#content > div > div.view-content > div > table

a_list = html.select('#content > div > div.view-content > div > table > tbody > tr > td > a')
print(len(a_list))
a_list[:4]

50


[<a href="/civilappeal/25670204">다자녀가정 실내 바닥매트 지원</a>,
 <a href="/civilappeal/23194045">[서울산업진흥원] 서울메이드란?</a>,
 <a href="/civilappeal/22904492">광진맘택시 운영(임산부,영아 양육가정 전용 택시)</a>,
 <a href="/civilappeal/22477798">마포 뇌병변장애인 비전센터</a>]

In [77]:
df_table = table[0]
print(df_table.shape)
df_table.head()

(50, 4)


,번호,제목,생산일,조회수
0,2470,다자녀가정 실내 바닥매트 지원,2021-08-17,155
1,2469,[서울산업진흥원] 서울메이드란?,2021-06-29,675
2,2468,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,712
3,2467,마포 뇌병변장애인 비전센터,2021-03-12,840
4,2466,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1177


In [78]:
# a_link_no 를 "내용번호" 컬럼을 생성해서 넣어줍니다.
# table["내용번호"]

a_link_no = pd.Series(a_list).apply(lambda x: int(x['href'].split('/')[-1])).values
df_table["내용번호"] = a_link_no

print(df_table.shape)
df_table.head()

(50, 5)


,번호,제목,생산일,조회수,내용번호
0,2470,다자녀가정 실내 바닥매트 지원,2021-08-17,155,25670204
1,2469,[서울산업진흥원] 서울메이드란?,2021-06-29,675,23194045
2,2468,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,712,22904492
3,2467,마포 뇌병변장애인 비전센터,2021-03-12,840,22477798
4,2466,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1177,21212235


In [79]:
a_link_no[:4]

array([25670204, 23194045, 22904492, 22477798], dtype=int64)

## 특정 페이지를 수집하는 함수 만들기

In [106]:
def get_one_page(page_no):
    header = {'user_agent':'Mozilla/5.0'}
    url = f"https://opengov.seoul.go.kr/civilappeal/list?items_per_page=50&page={page_no}"
    
    # table
    table = pd.read_html(base_url, encoding='utf-8')[0]
    
    # detail
    r = requests.get(url, headers = header)
    html = bs(r.text,'lxml')
    a_list = html.select('#content > div > div.view-content > div > table > tbody > tr > td > a')
    a_link_no = pd.Series(a_list).apply(lambda x : int(x['href'].split('/')[-1])).values
    table['내용번호'] = a_link_no
    
    return table

In [102]:
# 함수가 잘 동작하는지 확인
get_one_page(page_no=5)[:4]

,번호,제목,생산일,조회수,내용번호
0,2470,다자녀가정 실내 바닥매트 지원,2021-08-17,157,2896830
1,2469,[서울산업진흥원] 서울메이드란?,2021-06-29,675,2896279
2,2468,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,712,2898007
3,2467,마포 뇌병변장애인 비전센터,2021-03-12,840,2897591


In [104]:
# 없는 페이지도 확인
get_one_page(page_no=500)

<ipython-input-101-b16dd06cad61>:13: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  a_link_no = pd.Series(a_list).apply(lambda x : int(x['href'].split('/')[-1])).values


'500를 찾을 수 없습니다.'

## 반복문을 통한 여러 페이지 수집하기

* 게시물이 없을 때까지 수집하기

In [108]:
# time.sleep을 통해 일정 간격 쉬었다가 가져옵니다.
# 게시물이 없으면 멈춥니다.
page_no = 1
table_list = []

import time
page=1
while True:
    try :
        table = get_one_page(page)
        table_list.append(table)
        print(page,end=' ')
        page+=1
        time.sleep(0.01)
    except:
        break
len(table_list)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 

49

## 데이터 병합하기

In [109]:
# 여러 페이지의 내용을 하나로 병합합니다.
df = pd.concat(table_list)
df.reset_index(inplace=True, drop=True)
print(df.shape)
df

(2450, 5)


,번호,제목,생산일,조회수,내용번호
0,2470,다자녀가정 실내 바닥매트 지원,2021-08-17,157,25670204
1,2469,[서울산업진흥원] 서울메이드란?,2021-06-29,675,23194045
2,2468,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,712,22904492
3,2467,마포 뇌병변장애인 비전센터,2021-03-12,840,22477798
4,2466,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1177,21212235
...,...,...,...,...,...
2445,2425,작은연구 좋은서울 지원사업 공모,2016-08-10,436,2895717
2446,2424,어르신 돌봄가족 휴가제 (치매어르신 돌봄가족 지원),2016-08-09,6213,2895720
2447,2423,노인생애체험센터,2016-06-28,3906,2895632
2448,2422,아이조아 서울상 콘테스트,2016-06-10,855,2895719


## 데이터 파일로 저장하기

In [98]:
# 저장할 파일명
file_name = "seoul-120-list.csv"

In [99]:
# CSV 파일로 저장하기
df.to_csv(file_name,index=False)

In [112]:
# 저장한 파일 확인
read_csv = pd.read_csv(file_name)
print(read_csv.shape)
read_csv.head()

(2450, 5)


,번호,제목,생산일,조회수,내용번호
0,2470,다자녀가정 실내 바닥매트 지원,2021-08-17,156,25670204
1,2469,[서울산업진흥원] 서울메이드란?,2021-06-29,675,23194045
2,2468,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,712,22904492
3,2467,마포 뇌병변장애인 비전센터,2021-03-12,840,22477798
4,2466,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1177,21212235
